In [731]:
# импортируем библиотеки и модули
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import linear_model

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.ensemble import BaggingClassifier

In [732]:
# загружаем и визуализируем данные
data = pd.read_csv('train.csv', encoding = 'utf-8')
data.head( )

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [733]:
# Предварительная работа с данными 
columns_target = ['Survived'] # наша целевая колонка (относительно которой, мы будем предсказывать)
columns_train = ['Pclass', 'Sex', 'Age', 'Fare']

In [734]:
X = data[columns_train]
Y = data[columns_target]

In [735]:
X['Age'].isnull().sum() # !!!! 177 пропущенных значений

177

In [746]:
pd.options.mode.chained_assignment = None # отключаем розовые предупреждения)

In [737]:
# Заполняем пустые ячейки медианным значением по возрасту
X['Age'] = X['Age'].fillna(X['Age'].median( ))

In [738]:
X['Age'].isnull().sum() 

0

In [739]:
# Заменяем male и female на 0 и 1 с помощью словаря
d = {'male':0, 'female':1} # создаем словарь

In [740]:
X['Sex'] = X['Sex'].apply(lambda x:d[x]) # lambda означает, что мы зеркально заменяет значения в нашем словаре

In [741]:
X['Sex'].head() 

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int64

In [742]:
# Разделяем нашу выборку на обучающую и тестовую
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [743]:
clf = tree.DecisionTreeClassifier(max_depth =5, random_state = 21)

In [744]:
bagging = BaggingClassifier(base_estimator=clf,n_estimators=100)

In [747]:
bagging.fit(X_train,Y_train)

C:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5,
                                                        random_state=21),
                  n_estimators=100)

In [748]:
bagging.score (X_test,Y_test)

0.7966101694915254

In [749]:
clf.fit(X_train, Y_train) # обучаем модель

DecisionTreeClassifier(max_depth=5, random_state=21)

In [750]:
clf.score(X_test,Y_test) # проверяем точность

0.8169491525423729

In [751]:
import eli5

In [757]:
eli5.explain_weights_sklearn(clf, feature_names=X_train.columns.values)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 13: invalid start byte

Explanation(estimator='DecisionTreeClassifier(max_depth=5, random_state=21)', description='\nDecision tree feature importances; values are numbers 0 <= x <= 1;\nall values sum to 1.\n', error=None, method='decision tree', is_regression=False, targets=None, feature_importances=FeatureImportances(importances=[FeatureWeight(feature='Sex', weight=0.5264331520456753, std=None, value=None), FeatureWeight(feature='Pclass', weight=0.17952350256153152, std=None, value=None), FeatureWeight(feature='Fare', weight=0.17489242205608208, std=None, value=None), FeatureWeight(feature='Age', weight=0.11915092333671104, std=None, value=None)], remaining=0), decision_tree=TreeInfo(criterion='gini', tree=NodeInfo(id=0, is_leaf=False, value=[374.0, 222.0], value_ratio=[0.62751677852349, 0.3724832214765101], impurity=0.46747894238998244, samples=596, sample_ratio=1.0, feature_name='Sex', feature_id=1, threshold=0.5, left=NodeInfo(id=1, is_leaf=False, value=[317.0, 73.0], value_ratio=[0.8128205128205128, 0.18